In [1]:
%pip install evidently

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd

from evidently.dashboard import Dashboard
from evidently.dashboard.tabs import (
    DataDriftTab,
    DataQualityTab
)
from evidently import ColumnMapping
from evidently.options.data_drift import DataDriftOptions


prod = pd.read_csv('data/prod.csv')
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

options = DataDriftOptions()
dashboard = Dashboard(tabs=[DataQualityTab(), DataDriftTab()], options=[options])
dashboard.calculate(train, prod)
dashboard.save("reports/data_quality_drift.html")

In [4]:
prod

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,reg_prediction,class_prediction
0,9.8,0.310,0.49,15.4,0.046,13.0,119.0,1.00040,3.18,0.45,9.5,5,5.00,5
1,6.8,0.280,0.34,7.5,0.035,34.0,177.0,0.99692,3.33,0.43,9.1,5,5.57,6
2,5.7,0.335,0.34,1.0,0.040,13.0,174.0,0.99200,3.27,0.66,10.0,5,5.59,5
3,5.7,0.265,0.28,6.9,0.036,46.0,150.0,0.99299,3.36,0.44,10.8,7,6.63,6
4,5.3,0.360,0.27,6.3,0.028,40.0,132.0,0.99186,3.37,0.40,11.6,6,6.22,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,6.7,0.240,0.35,13.1,0.050,64.0,205.0,0.99700,3.15,0.50,9.5,5,5.83,6
976,7.4,0.180,0.27,1.3,0.048,26.0,105.0,0.99400,3.52,0.66,10.6,6,6.30,6
977,8.0,0.260,0.28,8.2,0.038,72.0,202.0,0.99566,3.12,0.56,10.0,6,5.81,6
978,5.9,0.270,0.32,2.0,0.034,31.0,102.0,0.98952,3.16,0.56,12.3,6,6.48,6


In [10]:

from evidently.dashboard.tabs import NumTargetDriftTab
from evidently.dashboard.tabs import RegressionPerformanceTab

In [12]:
column_mapping = ColumnMapping(target='quality', prediction='reg_prediction')
test_2 = test.drop(columns=['class_prediction'])
prod_2 = prod.drop(columns=['class_prediction'])
reg_drift_report = Dashboard(tabs=[NumTargetDriftTab(), RegressionPerformanceTab(verbose_level=1)])
reg_drift_report.calculate(test_2, prod_2, column_mapping=column_mapping)
reg_drift_report.save("reports/reg_drift.html")

In [35]:


from evidently.dashboard.tabs import CatTargetDriftTab
from evidently.dashboard.tabs import ProbClassificationPerformanceTab
import random

test_3 = test.drop(columns=['reg_prediction'])
prod_3 = prod.drop(columns=['reg_prediction'])
numerical_columns = test_3.columns.tolist()
numerical_columns.remove('quality')
numerical_columns.remove('class_prediction')
column_mapping = ColumnMapping(target='quality', prediction=["class_0", "class_1"], numerical_features=numerical_columns)

prod_3['class_prediction'] = [random.uniform(0, 1) for _ in range(len(prod_3))]
prod_3['quality'] = [float(random.choice([0, 1])) for _ in range(len(prod_3))]
test_3['class_0'] = [random.uniform(0, 1) for _ in range(len(test_3))]
test_3['class_1'] = 1 - test_3['class_0']
test_3['quality'] = [float(random.choice([0, 1])) for _ in range(len(test_3))]
data_and_target_drift_report = Dashboard(tabs=[ProbClassificationPerformanceTab(verbose_level=0)])
data_and_target_drift_report.calculate(test_3, prod_3, column_mapping = column_mapping)
data_and_target_drift_report.save("reports/classification.html")

/Users/andrebajorat/Library/Python/3.8/lib/python/site-packages/evidently/analyzers/prob_classification_performance_analyzer.py:82: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



AttributeError: 'bool' object has no attribute 'astype'

In [36]:
prod_3

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,class_prediction
0,9.8,0.310,0.49,15.4,0.046,13.0,119.0,1.00040,3.18,0.45,9.5,1.0,0.058312
1,6.8,0.280,0.34,7.5,0.035,34.0,177.0,0.99692,3.33,0.43,9.1,1.0,0.664737
2,5.7,0.335,0.34,1.0,0.040,13.0,174.0,0.99200,3.27,0.66,10.0,1.0,0.234555
3,5.7,0.265,0.28,6.9,0.036,46.0,150.0,0.99299,3.36,0.44,10.8,1.0,0.113422
4,5.3,0.360,0.27,6.3,0.028,40.0,132.0,0.99186,3.37,0.40,11.6,0.0,0.458869
...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,6.7,0.240,0.35,13.1,0.050,64.0,205.0,0.99700,3.15,0.50,9.5,0.0,0.086501
976,7.4,0.180,0.27,1.3,0.048,26.0,105.0,0.99400,3.52,0.66,10.6,0.0,0.527735
977,8.0,0.260,0.28,8.2,0.038,72.0,202.0,0.99566,3.12,0.56,10.0,1.0,0.728218
978,5.9,0.270,0.32,2.0,0.034,31.0,102.0,0.98952,3.16,0.56,12.3,1.0,0.035493


In [37]:
from sklearn import datasets
iris = datasets.load_iris()

In [28]:
test_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed_acidity         392 non-null    float64
 1   volatile_acidity      392 non-null    float64
 2   citric_acid           392 non-null    float64
 3   residual_sugar        392 non-null    float64
 4   chlorides             392 non-null    float64
 5   free_sulfur_dioxide   392 non-null    float64
 6   total_sulfur_dioxide  392 non-null    float64
 7   density               392 non-null    float64
 8   pH                    392 non-null    float64
 9   sulphates             392 non-null    float64
 10  alcohol               392 non-null    float64
 11  quality               392 non-null    int64  
 12  class_prediction      392 non-null    float64
dtypes: float64(12), int64(1)
memory usage: 39.9 KB


In [39]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')